# Introducción 

En este notebook voy a documentar el proceso de desarrollo del proyecto en cuanto a código se refiere. Nuestra intención es lograr que un vehiculo equipado con una Raspberry Pi 3 sea capaz de seguir una trazada autónomamente. 
Para lograr esto, primeramente se han recogido muestras de la trazada con la webcam que incluye este vehiculo.
Estas muestras van a pasar por un preprocesamiento previo para reducir las componentes, para más tarde usar estas componentes en una red neuronal multicapa o MLP. 


# Preprocesado

En esta sección nuestra intención es la de preparar los datos de entrada a la red con el fin de hacerlos lo más sencillos posible de procesar por nuestra red. Esto se traducirá en una binarización de la imagen, aislando de esta manera la información útil, es decir, la trazada a seguir.
Para este caso, se han tomado imágenes con una webcam conectada al vehiculo, que nos proporciona capturas de 640 de ancho y 480 de alto. Esto nos daría un total de 921600 entradas a la red, excesivo en cualquier caso.
Dado que el número de datos de entrenamiento aumenta con el número de entradas, necesitaríamos una cantidad inmensa de muestras, con todo lo que eso conlleva: elevado tiempo de cómputo, red compleja, etc.
Por lo mencionado anteriormente, el preprocesado de los datos de entrada es de carácter obligatorio.
Las imágenes usadas se componen de un fondo gris con la línea azul a seguir, como se muestra en este ejemplo:

In [1]:
921600 import cv2
import numpy as np

img = cv2.imread('imagen_2021-02-16 14_47_49.661111_EjeIzda1_-0.762908935546875_EjeDcha4_-0.979400634765625.jpg')
cv2.imshow('Imagen Original', img)
cv2.waitKey(0)

-1

Como se puede apreciar, hay una gran cantidad de píxeles que no aportan apenas información pero sí que obstaculizan el entrenamiento.
Estas imágenes pasaran por una serie de procesos hasta obtener una imagen binarizada, redimensionada y normalizada.
Se ha usado un dataset de 6312 imágenes, suficientes para producir un bajo error. 

### Desarrollo del código

En un fichero llamado preprocesar.py se ha diseñado un código simple pero que ejecuta a la perfección esta necesidad.
En primer lugar, importamos los paquetes necesarios, estos son: *os*, *cv2*, *numpy* y *re*.

In [2]:
import os
import cv2
import numpy as np
import re

La idea es la de "muestrear" la imagen y convertirla en un vector fila con todos los valores de los pixeles finales. Para ello se crean las variables usadas como índices del array que contendrá dichos valores. A continuación, se recupera el número de imágenes disponibles en el directorio.

In [ ]:
indice_x = 0
indice_y = 0

dir = 'D:/imagenes'

path, dirs, files = next(os.walk(dir))
file_count = len(files)

"Escaneamos" el directorio en busca de las imágenes, que renombramos como *imagenes*. Creamos una matriz vacía en la que guardaremos los valores de los pixeles seleccionados. Como se puede observar, tiene tantas filas como elementos en el directorio y tantas columnas como número total de puntos que queramos o dicho de otra manera, el número final de píxeles a los que reduciremos la imagen añadiendole dos columnas extra, dedicadas a los valores de salida para ese conjunto. 

In [ ]:
with os.scandir(dir) as imagenes:
    valores = np.empty((file_count, 24 * 18 + 2))

Prosigue el bucle principal, donde se van a realizar las operaciones. 
Para poner en contexto, los archivos estan nombrados de manera que aparezca la fecha y hora de la toma, así como los valores que tenían los joysticks del mando. Será necesario extraer estos últimos dos valores, pues se tratan de nuestras salidas.
Sabiendo esto, se ha usado una *expresion regular*. Para este caso, usando el paquete *re* y su función *findall* se encuentran todos los números existentes en el nombre.
Continúa cargando la imagen y comenzando el procesado:

* Aumento del contraste
* Conversión a escala de grises
* Binarización
* Redimensionado 
* Normalización entre 0 y 1

Se extraen los valores del eje izquierdo y derecho y comienza el "muestreo" de la imagen.

Esta operación puede llevar unos minutos de cómputo, dependiendo del número de imágenes y el tamaño final deseado. En un equipo de 8 núcleos y 16 hilos y 16GB de memoria RAM tarda aproximadamente menos de un minuto.

In [5]:
    for imagen in imagenes:
        valoresSalida = [float(s) for s in re.findall(r'-?\d+\.?\d*', imagen.name)]
        img = cv2.imread(dir + '/' + imagen.name)
        contrast_img = cv2.addWeighted(img, 2.3, np.zeros(img.shape, img.dtype), 0, 0)
        gray_img = cv2.cvtColor(contrast_img, cv2.COLOR_BGR2GRAY)
        ret, thresh1 = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY_INV)
        imagen_re = cv2.resize(thresh1, (24, 18))
        final_img = imagen_re / 255
        valores[indice_x, indice_y] = valoresSalida[7]
        valores[indice_x, indice_y + 1] = valoresSalida[9]
        for i in range(final_img.shape[0]):
            for j in range(final_img.shape[1]):
                valores[indice_x, indice_y + 2] = final_img[i, j]
                indice_y = indice_y + 1
        indice_x = indice_x + 1
        indice_y = 0

    valores = np.round(valores, 3)

NameError: name 'imagenes' is not defined

El resultado de aplicar los dintintos procesamientos se muestran a continuación:

#### Cargar imagen

In [13]:
img = cv2.imread('imagen_2021-02-16 14_47_49.661111_EjeIzda1_-0.762908935546875_EjeDcha4_-0.979400634765625.jpg')
cv2.imshow('Imagen Original', img)
cv2.waitKey(0)

-1

#### Aumento del contraste

In [3]:
contrast_img = cv2.addWeighted(img, 2.3, np.zeros(img.shape, img.dtype), 0, 0)
cv2.imshow('Imagen Contrastada', contrast_img)
cv2.waitKey(0)

-1

#### Conversión a escala de grises

In [4]:
gray_img = cv2.cvtColor(contrast_img, cv2.COLOR_BGR2GRAY)
cv2.imshow('Imagen en Gris', gray_img)
cv2.waitKey(0)

-1

#### Binarización

In [5]:
ret, thresh1 = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('Imagen Binarizada', thresh1)
cv2.waitKey(0)

-1

#### Redimensionado

In [12]:
resize_img = cv2.resize(thresh1, (24, 18))
cv2.imshow('Imagen Redimensionada', resize_img)
cv2.waitKey(0)

-1

Para finalizar, guardamos el array con los valores capturados en formato *csv* para poder usarlo posteriormente.

In [ ]:
np.savetxt('data.csv', valores, delimiter=',')

# Entrenamiento

En la sección anterior se ha creado un dataset listo para ser usado por una red neuronal. En esta, se verá la creacion de la misma, los resultados obtenidos y el por qué de las decisiones tomadas.
Se ha decidido usar una Red Neuronal Artificial (ANN), más concretamente una Multilayer Perceptron (MLP) por su gran desempeño y facilidad de creación.
Dado que no son valores discretos, como podría ser la pertenencia a una clase o etiqueta, sino que son valores continuos, estamos ante un problema de regresión. El fin de la red será dado un vector, ser capaz de entregar de la manera más fiel posible los valores que un piloto usaría con el mando ante esa misma situación. Se obtendrán valores entre 0 y -1, aunque no se tratan de límites estrictos.
Para ello, se ha usado la librería *tensorflow* que incluye *keras*. Más adelante se comentará con mayor detalle.

### Desarrollo del código

En un fichero llamado "entrenar.py" se ha diseñado la red. Comenzamos importando todas las funciones necesarias, estas son:

* Sequential: Agrupa una pila de capas a un modelo

* Dense: Capa de neuronas

* Dropout: Desactiva un tanto por ciento de neuronas

* Adam: Optimizador del modelo

* train_test_split: Divide un dataset aleatoriamente en conjunto de entrenamiento y conjunto de test

* EarlyStopping: Para automáticamente cuando no se mejora el error de validación

* mean_squared_error: Usado para calcular el error entre el conjunto de test y las predicciones del modelo

Usando el dataset generado en la sección anterior, cargamos los valores que se van a trabajar.


In [ ]:
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import os
import matplotlib.pyplot as plt


dataframe = read_csv("data.csv", header=None, sep=',')
dataset = dataframe.values

El dataset cargado se divide en en valores de entrada "X" y valores de salida "Y". A su vez, estos dataset se dividen usando la función anteriormente mencionada, creando un conjunto de entrenamiento del 80% de las muestras, dejando un 10% para el conjunto de validación y otro 10% para el conjunto de test.

In [ ]:
X = dataset[:, 2:]
Y = dataset[:,0:2]

lr = 0.001
filas_entrada = dataset.shape[0]
columnas_entrada = dataset.shape[1]-2
batch_size = 32

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5)

Siguiendo con lo anterior, se puede dar comienzo al diseño de la red neuronal. Para ello, creamos un objeto tipo *Sequential*, que será el plano donde se añadirán las capas de neuronas.
Será una red de 3 capas, una capa de entrada y 2 capas ocultas, por lo que se puede considerar una red profunda. Este es el diseño que mejor resultado ha obtenido para este trabajo.
Continuamos añadiendo la primer capa densa, con tantas neuronas como características de entrada (de aquí la importancia de contar con un buen procesado de datos), con una función de activación tipo "relu". A esta capa le añadiremos un *Dropout* del 40%.
Agregamos la primera capa oculta, con un tercio de neuronas que la capa de entrada, cuya función de activacion es la misma, al igual que el *Dropout*.
La segunda capa oculta cuenta con un quinto de neuronas que la capa de entrada, manteniendo la función de activacion y bajando la probabilidad del *Dropout* al 30%.
Terminamos el diseño con la capa de salida, que consta de 2 neuronas ya que son necesarios dos valores para el funcionamiento del vehiculo. Dado que se trata de un problema de regresión, la función de activación de la capa de salida debe ser lineal.
Nótese que lo que se busca con este diseño es un embotellamiento, comenzando por una mayor cantidad de neuronas para ir reduciendo el número capa a capa. Dado que son capas densas, están interconectadas entre ellas, o dicho de otra manera, es una red full conected.

In [ ]:
red = Sequential()

red.add(Dense(units=columnas_entrada, activation='relu'))
red.add(Dropout(0.4))

red.add(Dense(units=int(columnas_entrada/3), activation='relu'))
red.add(Dropout(0.4))

red.add(Dense(units=int(columnas_entrada/5), activation='relu'))
red.add(Dropout(0.3))

red.add(Dense(units=2, activation='linear'))

Una vez diseñada la red, debemos definir qué se quiere medir, cómo se va a mejorar y qué medida juzgará el desempeño.
Dado que es un problema de regresión, las pérdidas se mediran con el error cuadrático medio o mse, se optimizará o actualizará el valor de los pesos con *Adam* y observaremos la evolución del error cuadrático medio.
También iniciamos el *EarlyStopping*, simplemente indicando cuantas épocas deben pasar sin mejoría para detener el entrenamiento. Además, le indicamos que una vez finalizado el entrenamiento, recupere la mejor configuración que haya sido capaz de encontrar. Puede parecer que el número de épocas sin mejoría es alto, pero se ha indicado un learning rate bajo, por lo que los pesos variarán lentamente. De esta manera se asegura un resultado competitivo.
Por último, comienza el entrenamiento. Será necesario indicar el conjunto de entrenamiento de entrada con sus salidas correspondientes, un batch_size de 32 muestras, los datos para la validación, creados anteriormente y un callback, donde añadiremos el *EarlyStopping*.

Esta operación puede llevar tiempo de procesamiento, dependiendo de la capacidad de la máquina usada, el tamaño de los datos, el batch_size, etc. 

In [ ]:
red.compile(loss='mean_squared_error', optimizer=Adam(lr=lr), 
            metrics=['MeanSquaredError'])

early_stopping = EarlyStopping(patience=200, restore_best_weights=True)

resultado = red.fit(x=X_train, y=Y_train, batch_size=batch_size,
            epochs=10000, validation_data=(X_val, Y_val),
            verbose=2, callbacks=[early_stopping])

A modo de comprobación, se puede mostrar el mejor valor del error de validación, el que podemos comparar con el error que se genera entre la predicción y el conjunto de test. En la gran mayoria de casos, este segundo error debe ser mínimamente mayor que el error de validación, lo que nos indica que es capaz de predecir correctamente. Se generan dos gráficas pertenecientes a la evolución de las pérdidas de entrenamiento y de validación.

In [ ]:
print(min(resultado.history['val_loss']))

Y_pred = red.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)

print(mse)

plt.style.use("ggplot")
plt.figure()
plt.plot(resultado.history['loss'], label="train_loss")
plt.title("Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.figure()
plt.plot(resultado.history['val_loss'], label="val_loss")
plt.title("Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

Para finalizar esta sección, creamos un directorio donde guardar la configuración calculada por la red y guardamos tanto el modelo como los pesos, que usaremos más adelante.

In [ ]:
dir = 'modelo'

if not os.path.exists(dir):
    os.mkdir(dir)
red.save('modelo/modelo.h5')
red.save_weights('modelo/pesos.h5')

# Test

Antes de probar estos resultados en el vehiculo, se va a simular el comportamiento del modelo entrenado. 
Para ello, se creará una funcion que realice el preprocesado de la imagen, y con una imagen que no se haya usado para el entrenamiento, se probará.
Se carga el modelo y sus pesos correspondientes.

In [5]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import re

modelo = 'modelo.h5'
pesos = 'pesos.h5'
red = load_model(modelo)
red.load_weights(pesos)

La función *predict* será la encargada de dicha tarea. Es el mismo procesado que sufren las imágenes para generar el dataset.

In [6]:
def predict(file):
    indice = 0
    valoresSalida = [float(s) for s in re.findall(r'-?\d+\.?\d*', file)]
    img = cv2.imread(file)
    valores = np.empty((1, 24*18))
    contrast_img = cv2.addWeighted(img, 2.3, np.zeros(img.shape, img.dtype), 0, 0)
    gray_img = cv2.cvtColor(contrast_img, cv2.COLOR_BGR2GRAY)
    ret, thresh1 = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY_INV)
    resize_img = cv2.resize(thresh1, (24, 18))
    final_img = resize_img / 255
    for i in range(final_img.shape[0]):
        for j in range(final_img.shape[1]):
            valores[0, indice] = final_img[int(i), int(j)]
            indice = indice + 1


    respuesta = red.predict(valores)
    print('Eje izquierdo real: ',valoresSalida[7], 
          ', Eje derecho real: ', valoresSalida[9])
    print('Eje izquierdo predicho: ',respuesta[0,0], 
          ', Eje derecho predicho: ', respuesta[0,1])

Hacemos uso de la función, y como se puede ver, el error entre ambos valores es mínimo, por lo que se puede esperar que el funcionamiento del vehiculo en las pruebas sea satisfactoria.

In [7]:
predict('imagen_2021-02-16 14_47_49.661111_EjeIzda1_-0.762908935546875_EjeDcha4_-0.979400634765625.jpg')

Eje izquierdo real:  -0.762908935546875 , Eje derecho real:  -0.979400634765625
Eje izquierdo predicho:  -0.7943145 , Eje derecho predicho:  -1.0023944


array([[-0.7943145, -1.0023944]], dtype=float32)

# Prueba en el vehiculo

Ahora que hemos comprobado que el regresor funciona correctamente, podemos cargarlo en el vehiculo y comenzar las pruebas reales.
En primer lugar cargamos los paquetes necesarios, el modelo y los pesos calculados en el entrenamiento.

In [ ]:
import RPi.GPIO as GPIO
import numpy as np
from tensorflow.keras.models import load_model
import cv2


# Cargamos el modelo y los pesos de la red
modelo = 'modelo.h5'
pesos = 'pesos.h5'

red = load_model(modelo)
red.load_weights(pesos)

Configuramos los pines del puerto GPIO de la Raspberry Pi, describiendolos como salidas que proporcionarán un pulso PWM a los motores para así regular la velocidad. Hacemos esto para el eje derecho e izquierdo. Finalmente, comenzamos a capturar imagenes con la cámara.

In [6]:
directionRight = 31 #Conectar a D4
speedRight = 33 # PWM pin connected Right motor #Conectar a M1 -> D5
speedLeft = 35 # PWM pin connected Left motor #Conectar a M2 -> D6
directionLeft = 37 #Conectar a D7

GPIO.setwarnings(False) #disable warnings
GPIO.setmode(GPIO.BOARD) #set pin numbering system # Referencia a la posición física.

GPIO.setup(directionLeft,GPIO.OUT)
GPIO.setup(directionRight,GPIO.OUT)

GPIO.setup(speedLeft,GPIO.OUT)
pwmLeft = GPIO.PWM(speedLeft,100) #create PWM instance with frequency #En el código de ejemplo usa 1000, pero no me va bien.
pwmLeft.start(0) #start PWM of required Duty Cycle

GPIO.setup(speedRight,GPIO.OUT)
pwmRight = GPIO.PWM(speedRight,100) #create PWM instance with frequency
pwmRight.start(0) #start PWM of required Duty Cycle

cam = cv2.VideoCapture(0)


NameError: name 'GPIO' is not defined

Entramos en un bucle infinito, pues el coche no para de moverse hasta que sea interrumpido. Leemos la imagen capturada y se le aplica el procesado explicado anteriormente. Extraemos los valores de la predicción y comprobamos su polaridad. Si el resultado es mayor que cero significa que las ruedas deberán girar en sentido contrario, es decir, marcha atras. Por otro lado, si es menos que cero, girarán en el sentido de la circulación.
Para regular la potencia, definimos la variable *pot* que ocupará un valor entre 0 y 100. Este valor, multiplicado con la salida de la red indicará el tamaño del pulso PWM, en otras palabras, el tiempo que estará encendido el motor en un ciclo del pulso.
Para terminar, liberamos la cámara cuando se detenga el código.

In [ ]:
while (True):
    ret, img = cam.read()
    indice = 0
    valores = np.empty((1, 24*18))
    contrast_img = cv2.addWeighted(img, 2.3, np.zeros(img.shape, img.dtype), 0, 0)
    gray_img = cv2.cvtColor(contrast_img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_img, 200, 255, cv2.THRESH_BINARY_INV)
    resize_img = cv2.resize(thresh, (24,18))
    final_img = resize_img / 255
    
    for i in range(final_img.shape[0]):
        for j in range(final_img.shape[1]):
            valores[0, indice] = final_img[i, j]
            indice = indice + 1
            
    output = red.predict(valores)
    Eje_izq = output[0,0]
    Eje_dch = output[0,1]
    
    if Eje_izq < 0:
        GPIO.output(directionLeft, True)
    else:
        GPIO.output(directionLeft, False)
        
    if Eje_dch < 0:
        GPIO.output(directionRight, True)
    else:
        GPIO.output(directionRight, False)
        
    pot = 50 # JK pot = 100 va a toda velocidad. 
    pwmRight.ChangeDutyCycle(int (abs(Eje_dch)*pot))
    pwmLeft.ChangeDutyCycle(int (abs(Eje_izq)*pot))    

cam.release()